## Peer-graded Assignment week 3, notebook 1 of 3: Segmenting and Clustering Neighborhoods in Toronto

This notebook is used for assignment week 3, Datascience Capstone.

Download/inslall the required packages:

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Scrape the table of postal codes from following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, and transform into a pandas dataframe:

In [2]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table=soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])

Double-check that the table contains required columns, number of rows is OK and the content also looks OK:

In [3]:
df.describe()

,Postcode,Borough,Neighbourhood
count,288,288,288
unique,180,12,209
top,M8Y,Not assigned,Not assigned
freq,8,77,78


In [4]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df.tail()

,Postcode,Borough,Neighbourhood
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor
287,M9Z,Not assigned,Not assigned


Count rows cells with a borough that is Not assigned:

In [6]:
(df['Borough']=='Not assigned').sum()

77

Create new dataframe (df1) where rows with "Not assigned" in column "Borough" are dropped:

In [7]:
df1=df[df.Borough != 'Not assigned']

Double-checking that it worked:

In [8]:
(df1['Borough']=='Not assigned').sum()

0

Double-check that number of rows in df1 is as expected (288-77=211):

In [9]:
df1.shape

(211, 3)

Create one more dataframe (df2) and combine multiple neighbourhoods in one row in they belong to the same postal code area. After that double-check that it looks OK by looking at first 5 rows:

In [10]:
df2=df1.groupby(["Postcode","Borough"])['Neighbourhood'].apply(','.join).reset_index()
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Double-check that there are no cases where neighbourhood is assigned while borough is not assigned. Note that this control is done using the original data set (called 'df' in this assignment), making it possible to identify errors before any modifications.

In [11]:
df[(df['Borough']=='Not assigned') & (df['Neighbourhood']!='Not assigned')]

,Postcode,Borough,Neighbourhood


Identifying rows where neighbourhood needs to be updated:

In [12]:
df2[(df2['Borough']!='Not assigned') & (df2['Neighbourhood']=='Not assigned')]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


Replace not assigned neighborhoods with the corresponding borough:

In [13]:
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned', df2['Borough'], df2['Neighbourhood'])

Double-check that it worked by repeating the same step as above (no rows should be found if everything is OK):

In [14]:
df2[(df2['Borough']!='Not assigned') & (df2['Neighbourhood']=='Not assigned')]

,Postcode,Borough,Neighbourhood


Print the number of rows of my dataframe by using the .shape method:

In [15]:
print('Number of rows in the dataframe is:',df2.shape[0])

Number of rows in the dataframe is: 103
